In [1]:
import config
import tensorflow as tf
tf.app.flags.DEFINE_string('f', '', 'kernel')
from collections import deque
import model
from dataUtils import *
from logger import MyLogger
import sys
import PTB_data_reader
import time
import numpy as np
import lstm_char_cnn
import pickle
import dataloader
tf.logging.set_verbosity(tf.logging.ERROR)


logger = MyLogger("ERDMain")

W0830 21:49:09.941854 140607751878464 deprecation_wrapper.py:119] From /home/hadoop/ERD/model.py:6: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

Using TensorFlow backend.


In [2]:
# load twitter data
# load_data(FLAGS.data_file_path)
load_data_fast()

#load PTB data
# word_vocab, char_vocab, word_tensors, char_tensors, max_word_length = \
#     PTB_data_reader.load_data(FLAGS.data_dir, FLAGS.max_word_length, char_vocab, eos=FLAGS.EOS)
word_vocab, char_vocab, word_tensors, char_tensors = \
    PTB_data_reader.load_data_fast()
max_word_length = FLAGS.max_word_length
train_reader = PTB_data_reader.DataReader(word_tensors['train'], char_tensors['train'],
                          FLAGS.batch_size, FLAGS.max_sent_len) 

#load sentiment analysis data
sentiReader = dataloader.SentiDataLoader(
                                        dirpath = '/home/hadoop/trainingandtestdata',
                                        trainfile = 'training.1600000.processed.noemoticon.csv', 
                                        testfile = 'testdata.manual.2009.06.14.csv', 
                                        charVocab = char_vocab
                        )
# sentiReader.load_data()
sentiReader.load_data_fast(
                        '/home/hadoop/ERD/data/senti_train_data.pickle',
                        '/home/hadoop/ERD/data/senti_train_label.pickle',
                        '/home/hadoop/ERD/data/senti_test_data.pickle',
                        '/home/hadoop/ERD/data/senti_test_label.pickle'
                          )


# (self, input_dim, hidden_dim, max_seq_len, max_word_num, class_num, action_num):
print(  FLAGS.embedding_dim, FLAGS.hidden_dim, 
            FLAGS.max_seq_len, FLAGS.max_sent_len, 
                FLAGS.class_num, FLAGS.action_num   )
logger.info(    (FLAGS.embedding_dim, FLAGS.hidden_dim, 
                    FLAGS.max_seq_len, FLAGS.max_sent_len, 
                        FLAGS.class_num, FLAGS.action_num)  )

print(get_curtime() + " Data loaded.")
logger.info(get_curtime() + " Data loaded.")

max_sent: 31 ,  max_seq_len: 101
5802 data loaded


I0830 21:50:19.443750 140607751878464 logger.py:24] (300, 200, 101, 31, 2, 2)
I0830 21:50:19.447273 140607751878464 logger.py:24] 2019-08-30 21:50:19 Data loaded.


300 200 101 31 2 2
2019-08-30 21:50:19 Data loaded.


In [3]:
# # save the Twitter data
# data = get_data()
# with open('data/data_dict.txt', 'wb') as handle:
#     pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # save the PTB data
# with open('data/char_tensors.txt', 'wb') as handle:
#     pickle.dump(char_tensors, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/word_tensors.txt', 'wb') as handle:
#     pickle.dump(word_tensors, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('data/char_vocab.txt', 'wb') as handle:
#     pickle.dump(char_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/word_vocab.txt', 'wb') as handle:
#     pickle.dump(word_vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

# save the senti data
# with open('data/senti_train_data.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/senti_train_label.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.train_label, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('data/senti_test_data.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data/senti_test_label.pickle', 'wb') as handle:
#     pickle.dump(sentiReader.test_label, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [4]:
import importlib
importlib.reload(lstm_char_cnn)
importlib.reload(model)

<module 'model' from '/home/hadoop/ERD/model.py'>

In [5]:
w2v = lstm_char_cnn.WordEmbedding(
                max_word_length = FLAGS.max_char_num , 
                char_vocab_size = char_vocab.size, 
                char_embed_size = FLAGS.char_embed_size, 
                kernels = eval(FLAGS.kernels), 
                kernel_features = eval(FLAGS.kernel_features), 
                num_highway_layers = FLAGS.highway_layers,
                embedding_dim = FLAGS.embedding_dim
            )
lstm_lm = lstm_char_cnn.LSTM_LM(
            batch_size = FLAGS.batch_size, 
            num_unroll_steps = FLAGS.max_sent_len, 
            rnn_size = FLAGS.embedding_dim, 
            num_rnn_layers = FLAGS.rnn_layers, 
            word_vocab_size = word_vocab.size
        )

char_train_graph = lstm_char_cnn.infer_train_model(
                    w2v, lstm_lm, 
                    batch_size = FLAGS.batch_size, 
                    num_unroll_steps = FLAGS.max_sent_len, 
                    max_word_length = FLAGS.max_char_num, 
                    learning_rate = FLAGS.learning_rate,
                    max_grad_norm = FLAGS.max_grad_norm
                 )
#sentiment analysis model
s_model = model.SentiModel(FLAGS.hidden_dim, 5)
senti_train_graph = model.InferSentiTrainGraph(
                        w2v, 
                        lstm_lm, 
                        s_model, 
                        max_word_num = FLAGS.max_sent_len, 
                        max_char_num = FLAGS.max_char_num, 
                        hidden_dim = FLAGS.hidden_dim, 
                        sent_num = FLAGS.sent_num,
                        embedding_dim = FLAGS.embedding_dim
                    )

input_: Tensor("input:0", shape=(20, 31, 21), dtype=int32)
input_cnn: Tensor("Embedding_1/CNN_OUT/add_7:0", shape=(620, 1100), dtype=float32)
input_: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
input_cnn: Tensor("Embedding_2/CNN_OUT/add_7:0", shape=(?, 1100), dtype=float32)


In [3]:
def test(reader):
    return reader.iter()

In [32]:
def Train_Char_Model(session, train_model, train_reader, saver, summary_writer):
    best_valid_loss = None
    rnn_state = session.run(train_model.initial_rnn_state)
    cnt = 10
#     for epoch in range(FLAGS.max_epochs):
    for epoch in range(1):
        epoch_start_time = time.time()
        avg_train_loss = 0.0
        count = 0
        for x, y in train_reader.iter():
            count += 1
            start_time = time.time()

            loss, _, rnn_state, gradient_norm, step, _ = session.run([
                train_model.loss,
                train_model.train_op,
                train_model.final_rnn_state,
                train_model.global_norm,
                train_model.global_step,
                train_model.clear_char_embedding_padding
            ], {
                train_model.input: x,
                train_model.targets: y,
                train_model.drop_out: 0.8,
                train_model.initial_rnn_state: rnn_state
            })

            summary = tf.Summary(value=[
                tf.Summary.Value(tag="step_train_loss", simple_value=loss),
                tf.Summary.Value(tag="step_train_perplexity", simple_value=np.exp(loss)),
            ])
            summary_writer.add_summary(summary, step)

            avg_train_loss += 0.05 * (loss - avg_train_loss)

            time_elapsed = time.time() - start_time

            if count % FLAGS.print_every == 0:
                print('%6d: %d [%5d/%5d], train_loss/perplexity = %6.8f/%6.7f secs/batch = %.4fs, grad.norm=%6.8f' % (step,
                                                        epoch, count,
                                                        train_reader.length,
                                                        loss, np.exp(loss),
                                                        time_elapsed,
                                                        gradient_norm))
                cnt += 1
                if cnt == 10:
                    break
        print('Epoch training time:', time.time()-epoch_start_time)
        save_as = '%s/epoch%03d_%.4f.model' % (FLAGS.train_dir, epoch, avg_train_loss)
        saver.save(session, save_as)
        print('Saved char model', save_as)

def TrainSentiModel(sess, saver, logger, train_model, senti_reader, train_batch, test_batch):
    train_iter = 100
    for t_epoch in range(1): 
        # for validation
        sum_acc = 0.0
        sum_loss = 0.0
        for t_iter in range(train_iter):
            data_X, data_Y = senti_reader.GetTrainingBatch(
                                            t_iter, 
                                            train_batch
                            )
            feed_dic = {
                        train_model.sent_x: data_X, 
                        train_model.sent_y: data_Y
            }
            _, step, loss, acc = sess.run(
                                        [train_model.sent_train_op, 
                                         train_model.sent_global_step, 
                                         train_model.sent_loss, 
                                         train_model.sent_acc], 
                                        feed_dic)
            sum_loss += loss
            sum_acc += acc
            if t_iter % 10 == 9:
                sum_loss = sum_loss / 10
                sum_acc = sum_acc / 10
                ret_acc = sum_acc
                print(get_curtime() + " Step: " + str(step) + " Training loss: " + str(sum_loss) + " accuracy: " + str(sum_acc))
                logger.info(get_curtime() + " Step: " + str(step) + " Training loss: " + str(sum_loss) + " accuracy: " + str(sum_acc))
                sum_acc = 0.0
                sum_loss = 0.0
                break
        # for validation
        sum_acc = 0.0
        sum_loss = 0.0
        for t_iter in range(100):
            data_X, data_Y = senti_reader.GetTestData(t_iter, test_batch)
            feed_dic = {train_model.sent_x: data_X, train_model.sent_y: data_Y}
            loss, acc = sess.run([train_model.sent_loss, train_model.sent_acc], feed_dic)
            sum_loss += loss
            sum_acc += acc    
        sum_loss = sum_loss / 100
        sum_acc = sum_acc / 100
        ret_acc = sum_acc
        print(get_curtime() + " Step: " + str(step) + " validation loss: " + str(sum_loss) + " accuracy: " + str(sum_acc))
        logger.info(get_curtime() + " Step: " + str(step) + " validation loss: " + str(sum_loss) + " accuracy: " + str(sum_acc))
        sum_acc = 0.0
        sum_loss = 0.0

        saver.save(sess, "df_saved/sent_model")
    
        

In [ ]:
# train char model
with tf.Graph().as_default() as g:
    with tf.Session(graph=g) as sess:
        w2v = lstm_char_cnn.WordEmbedding(
                        max_word_length = FLAGS.max_char_num , 
                        char_vocab_size = char_vocab.size, 
                        char_embed_size = FLAGS.char_embed_size, 
                        kernels = eval(FLAGS.kernels), 
                        kernel_features = eval(FLAGS.kernel_features), 
                        num_highway_layers = FLAGS.highway_layers,
                        embedding_dim = FLAGS.embedding_dim
                    )
        lstm_lm = lstm_char_cnn.LSTM_LM(
                    batch_size = FLAGS.batch_size, 
                    num_unroll_steps = FLAGS.max_sent_len, 
                    rnn_size = FLAGS.embedding_dim, 
                    num_rnn_layers = FLAGS.rnn_layers, 
                    word_vocab_size = word_vocab.size
                )

        char_train_graph = lstm_char_cnn.infer_train_model(
                            w2v, lstm_lm, 
                            batch_size = FLAGS.batch_size, 
                            num_unroll_steps = FLAGS.max_sent_len, 
                            max_word_length = FLAGS.max_char_num, 
                            learning_rate = FLAGS.learning_rate,
                            max_grad_norm = FLAGS.max_grad_norm
                         )
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=4)
        sess = tf.Session()
        with sess.as_default():
            sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter("testdir/", graph=sess.graph)
        Train_Char_Model(sess, char_train_graph, train_reader, saver, summary_writer)
        
with tf.Graph().as_default() as g:
    with tf.Session(graph=g) as sess:
        w2v = lstm_char_cnn.WordEmbedding(
                        max_word_length = FLAGS.max_char_num , 
                        char_vocab_size = char_vocab.size, 
                        char_embed_size = FLAGS.char_embed_size, 
                        kernels = eval(FLAGS.kernels), 
                        kernel_features = eval(FLAGS.kernel_features), 
                        num_highway_layers = FLAGS.highway_layers,
                        embedding_dim = FLAGS.embedding_dim
                    )
        lstm_lm = lstm_char_cnn.LSTM_LM(
                    batch_size = FLAGS.batch_size, 
                    num_unroll_steps = FLAGS.max_sent_len, 
                    rnn_size = FLAGS.embedding_dim, 
                    num_rnn_layers = FLAGS.rnn_layers, 
                    word_vocab_size = word_vocab.size
                )

        char_train_graph = lstm_char_cnn.infer_train_model(
                            w2v, lstm_lm, 
                            batch_size = FLAGS.batch_size, 
                            num_unroll_steps = FLAGS.max_sent_len, 
                            max_word_length = FLAGS.max_char_num, 
                            learning_rate = FLAGS.learning_rate,
                            max_grad_norm = FLAGS.max_grad_norm
                         )
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=4)
        sess.run(tf.global_variables_initializer())
        checkpoint = tf.train.get_checkpoint_state("testdir/")
        if checkpoint and checkpoint.model_checkpoint_path:
            saver.restore(sess, checkpoint.model_checkpoint_path)
        summary_writer = tf.summary.FileWriter("testdir/", graph=sess.graph)
        Train_Char_Model(sess, char_train_graph, train_reader, saver, summary_writer)

input_: Tensor("input:0", shape=(20, 31, 21), dtype=int32)
input_cnn: Tensor("Embedding_1/CNN_OUT/add_7:0", shape=(620, 1100), dtype=float32)
     5: 0 [    5/ 1499], train_loss/perplexity = 9.15730667/9483.4804688 secs/batch = 1.2378s, grad.norm=0.12444942
    10: 0 [   10/ 1499], train_loss/perplexity = 9.08647346/8834.9746094 secs/batch = 1.1685s, grad.norm=0.13129051


In [28]:
# reuse model
with tf.Graph().as_default() as g:
    with tf.Session(graph=g) as sess:
        w2v = lstm_char_cnn.WordEmbedding(
                        max_word_length = FLAGS.max_char_num , 
                        char_vocab_size = char_vocab.size, 
                        char_embed_size = FLAGS.char_embed_size, 
                        kernels = eval(FLAGS.kernels), 
                        kernel_features = eval(FLAGS.kernel_features), 
                        num_highway_layers = FLAGS.highway_layers,
                        embedding_dim = FLAGS.embedding_dim
                    )
        lstm_lm = lstm_char_cnn.LSTM_LM(
                    batch_size = FLAGS.batch_size, 
                    num_unroll_steps = FLAGS.max_sent_len, 
                    rnn_size = FLAGS.embedding_dim, 
                    num_rnn_layers = FLAGS.rnn_layers, 
                    word_vocab_size = word_vocab.size
                )

        char_train_graph = lstm_char_cnn.infer_train_model(
                            w2v, lstm_lm, 
                            batch_size = FLAGS.batch_size, 
                            num_unroll_steps = FLAGS.max_sent_len, 
                            max_word_length = FLAGS.max_char_num, 
                            learning_rate = FLAGS.learning_rate,
                            max_grad_norm = FLAGS.max_grad_norm
                         )
        val_list1 = tf.global_variables()
        saver = tf.train.Saver(val_list1, max_to_keep=4)
        sess.run(tf.variables_initializer(val_list1))
        checkpoint = tf.train.get_checkpoint_state("testdir/")
        if checkpoint and checkpoint.model_checkpoint_path:
            saver.restore(sess, checkpoint.model_checkpoint_path)
#         Train_Char_Model(sess, char_train_graph, train_reader, saver, summary_writer)
        #sentiment analysis model
        s_model = model.SentiModel(FLAGS.hidden_dim, 5)
        senti_train_graph = model.InferSentiTrainGraph(
                                w2v, 
                                lstm_lm, 
                                s_model, 
                                max_word_num = FLAGS.max_sent_len, 
                                max_char_num = FLAGS.max_char_num, 
                                hidden_dim = FLAGS.hidden_dim, 
                                sent_num = FLAGS.sent_num,
                                embedding_dim = FLAGS.embedding_dim
                            )
        
        uninitialized_vars = []
        for var in tf.global_variables():
            try:
                sess.run(var)
            except:
                uninitialized_vars.append(var)
                print("un_init_var:", var)
        print(unitialized_vars)
        sess.run(tf.variables_initializer(uninitialized_vars))
        summary_writer = tf.summary.FileWriter("testdir/", graph=sess.graph)
        Train_Char_Model(sess, char_train_graph, train_reader, saver, summary_writer)
        

input_: Tensor("input:0", shape=(20, 31, 21), dtype=int32)
input_cnn: Tensor("Embedding_1/CNN_OUT/add_7:0", shape=(620, 1100), dtype=float32)
input_: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
input_cnn: Tensor("Embedding_2/CNN_OUT/add_7:0", shape=(?, 1100), dtype=float32)
un_init_var: <tf.Variable 'SentiModel/conv2/kernel:0' shape=(5, 300, 200) dtype=float32_ref>
un_init_var: <tf.Variable 'SentiModel/conv2/bias:0' shape=(200,) dtype=float32_ref>
un_init_var: <tf.Variable 'dense/kernel:0' shape=(200, 3) dtype=float32_ref>
un_init_var: <tf.Variable 'dense/bias:0' shape=(3,) dtype=float32_ref>
un_init_var: <tf.Variable 'global_step_1:0' shape=() dtype=int32_ref>
un_init_var: <tf.Variable 'Embedding/char_embedding/Adagrad:0' shape=(72, 15) dtype=float32_ref>
un_init_var: <tf.Variable 'Embedding/TDNN/kernel_1/w/Adagrad:0' shape=(1, 1, 15, 50) dtype=float32_ref>
un_init_var: <tf.Variable 'Embedding/TDNN/kernel_1/b/Adagrad:0' shape=(50,) dtype=float32_ref>
un_init_var: <tf.Var

In [17]:
val_list2

[<tf.Variable 'Embedding/char_embedding:0' shape=(72, 15) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_1/w:0' shape=(1, 1, 15, 50) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_1/b:0' shape=(50,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_2/w:0' shape=(1, 2, 15, 100) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_2/b:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_3/w:0' shape=(1, 3, 15, 150) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_3/b:0' shape=(150,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_4/w:0' shape=(1, 4, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_4/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_5/w:0' shape=(1, 5, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_5/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_6/w:0' shape=(1, 6, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embeddin

In [16]:
list( filter(lambda var: var not in val_list1, val_list2) )
# val_list2

[<tf.Variable 'SentiModel/conv2/kernel:0' shape=(5, 300, 200) dtype=float32_ref>,
 <tf.Variable 'SentiModel/conv2/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(200, 3) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(3,) dtype=float32_ref>,
 <tf.Variable 'global_step_1:0' shape=() dtype=int32_ref>,
 <tf.Variable 'Embedding/char_embedding/Adagrad:0' shape=(72, 15) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_1/w/Adagrad:0' shape=(1, 1, 15, 50) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_1/b/Adagrad:0' shape=(50,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_2/w/Adagrad:0' shape=(1, 2, 15, 100) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_2/b/Adagrad:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_3/w/Adagrad:0' shape=(1, 3, 15, 150) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_3/b/Adagrad:0' shape=(150,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_4/

In [18]:
val_list1

[<tf.Variable 'Embedding/char_embedding:0' shape=(72, 15) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_1/w:0' shape=(1, 1, 15, 50) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_1/b:0' shape=(50,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_2/w:0' shape=(1, 2, 15, 100) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_2/b:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_3/w:0' shape=(1, 3, 15, 150) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_3/b:0' shape=(150,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_4/w:0' shape=(1, 4, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_4/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_5/w:0' shape=(1, 5, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_5/b:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'Embedding/TDNN/kernel_6/w:0' shape=(1, 6, 15, 200) dtype=float32_ref>,
 <tf.Variable 'Embeddin

In [ ]:
#sentiment analysis model
s_model = model.SentiModel(FLAGS.hidden_dim, 5)
senti_train_graph = model.InferSentiTrainGraph(
                        w2v, 
                        lstm_lm, 
                        s_model, 
                        max_word_num = FLAGS.max_sent_len, 
                        max_char_num = FLAGS.max_char_num, 
                        hidden_dim = FLAGS.hidden_dim, 
                        sent_num = FLAGS.sent_num,
                        embedding_dim = FLAGS.embedding_dim
                    )
# df model
rdm_model = model.RDM_Model(
                max_seq_len = FLAGS.max_seq_len, 
                max_word_num = FLAGS.max_sent_len, 
                embedding_dim = FLAGS.embedding_dim, 
                hidden_dim = FLAGS.hidden_dim
            )
rdm_train_graph = model.InferRDMTrainGraph(
                        w2v, lstm_lm, s_model, rdm_model, 
                        batchsize=FLAGS.batch_size,
                        max_seq_len = FLAGS.max_seq_len, 
                        max_word_num = FLAGS.max_sent_len, 
                        max_char_num = FLAGS.max_char_num, 
                        hidden_dim = FLAGS.hidden_dim, 
                        embedding_dim = FLAGS.embedding_dim,
                        class_num = FLAGS.class_num
                )

# rl model
cm_model = model.CM_Model(
                    max_word_num = FLAGS.max_sent_len, 
                    embedding_dim = FLAGS.embedding_dim, 
                    hidden_dim = FLAGS.hidden_dim, 
                    action_num = FLAGS.action_num
            )
cm_train_graph = model.InferCMTrainGraph(
                        w2v, s_model, rdm_model, cm_model, 
                        max_word_num = FLAGS.max_sent_len, 
                        embedding_dim = FLAGS.embedding_dim, 
                        hidden_dim = FLAGS.hidden_dim, 
                        action_num = FLAGS.action_num
                    )


In [ ]:

saver = tf.train.Saver(tf.global_variables(), max_to_keep=4)
sess = tf.Session()
with sess.as_default():
    sess.run(tf.global_variables_initializer())

summary_writer = tf.summary.FileWriter(FLAGS.train_dir, graph=sess.graph)

In [7]:
import importlib
importlib.reload(lstm_char_cnn)

<module 'lstm_char_cnn' from '/home/hadoop/ERD/lstm_char_cnn.py'>

In [ ]:
lstm_char_cnn.Train_Char_Model(sess, char_train_graph, train_reader, saver, summary_writer)

     5: 0 [    5/ 1499], train_loss/perplexity = 9.15729713/9483.3896484 secs/batch = 0.5930s, grad.norm=0.12490036
    10: 0 [   10/ 1499], train_loss/perplexity = 9.08498096/8821.7978516 secs/batch = 0.6202s, grad.norm=0.13333045
    15: 0 [   15/ 1499], train_loss/perplexity = 8.98046017/7946.2880859 secs/batch = 0.7101s, grad.norm=0.18570276
    20: 0 [   20/ 1499], train_loss/perplexity = 8.63869190/5645.9394531 secs/batch = 0.5504s, grad.norm=0.43776885
    25: 0 [   25/ 1499], train_loss/perplexity = 7.53612280/1874.5479736 secs/batch = 0.6260s, grad.norm=0.47992179
    30: 0 [   30/ 1499], train_loss/perplexity = 7.29143143/1467.6700439 secs/batch = 0.5361s, grad.norm=1.12251890
    35: 0 [   35/ 1499], train_loss/perplexity = 7.44433117/1710.1411133 secs/batch = 0.5397s, grad.norm=1.29667437
    40: 0 [   40/ 1499], train_loss/perplexity = 7.20252466/1342.8166504 secs/batch = 0.5403s, grad.norm=1.38146412
    45: 0 [   45/ 1499], train_loss/perplexity = 7.23605251/1388.6016846

   365: 0 [  365/ 1499], train_loss/perplexity = 6.63771820/763.3511963 secs/batch = 0.5380s, grad.norm=1.15613115
   370: 0 [  370/ 1499], train_loss/perplexity = 6.57711983/718.4670410 secs/batch = 0.5411s, grad.norm=0.44019940
   375: 0 [  375/ 1499], train_loss/perplexity = 6.43539858/623.5310669 secs/batch = 0.5414s, grad.norm=0.33791822
   380: 0 [  380/ 1499], train_loss/perplexity = 6.64196396/766.5990601 secs/batch = 0.5434s, grad.norm=0.52224058
   385: 0 [  385/ 1499], train_loss/perplexity = 6.61221409/744.1287842 secs/batch = 0.5400s, grad.norm=0.50121397
   390: 0 [  390/ 1499], train_loss/perplexity = 6.81604147/912.3662109 secs/batch = 0.5370s, grad.norm=0.35354772
   395: 0 [  395/ 1499], train_loss/perplexity = 6.80333662/900.8480835 secs/batch = 0.5420s, grad.norm=0.41359380
   400: 0 [  400/ 1499], train_loss/perplexity = 6.71813107/827.2699585 secs/batch = 0.5535s, grad.norm=0.49285099
   405: 0 [  405/ 1499], train_loss/perplexity = 6.58374310/723.2414551 secs/bat

   725: 0 [  725/ 1499], train_loss/perplexity = 6.52330303/680.8234863 secs/batch = 0.5357s, grad.norm=0.67174274
   730: 0 [  730/ 1499], train_loss/perplexity = 6.60094213/735.7880859 secs/batch = 0.5373s, grad.norm=0.45806772
   735: 0 [  735/ 1499], train_loss/perplexity = 6.57344103/715.8287964 secs/batch = 0.5337s, grad.norm=0.41944647
   740: 0 [  740/ 1499], train_loss/perplexity = 6.57568359/717.4359131 secs/batch = 0.5410s, grad.norm=0.30755678
   745: 0 [  745/ 1499], train_loss/perplexity = 6.48124552/652.7835083 secs/batch = 0.5372s, grad.norm=0.41048518
   750: 0 [  750/ 1499], train_loss/perplexity = 6.62926912/756.9287720 secs/batch = 0.5467s, grad.norm=0.34821105
   755: 0 [  755/ 1499], train_loss/perplexity = 6.50807333/670.5332642 secs/batch = 0.5429s, grad.norm=0.32785997
   760: 0 [  760/ 1499], train_loss/perplexity = 6.28847456/538.3314819 secs/batch = 0.5420s, grad.norm=0.33857116
   765: 0 [  765/ 1499], train_loss/perplexity = 6.53475618/688.6658325 secs/bat

  1085: 0 [ 1085/ 1499], train_loss/perplexity = 6.68645906/801.4792480 secs/batch = 0.5370s, grad.norm=0.32147256
  1090: 0 [ 1090/ 1499], train_loss/perplexity = 6.43140650/621.0468140 secs/batch = 0.5377s, grad.norm=0.34510538
  1095: 0 [ 1095/ 1499], train_loss/perplexity = 6.62986803/757.3822021 secs/batch = 0.5432s, grad.norm=0.32318753
  1100: 0 [ 1100/ 1499], train_loss/perplexity = 6.62116766/750.8212891 secs/batch = 0.5441s, grad.norm=0.29048184
  1105: 0 [ 1105/ 1499], train_loss/perplexity = 6.53903818/691.6210327 secs/batch = 0.5483s, grad.norm=0.34881800
  1110: 0 [ 1110/ 1499], train_loss/perplexity = 6.60372019/737.8349609 secs/batch = 0.5392s, grad.norm=0.28734133
  1115: 0 [ 1115/ 1499], train_loss/perplexity = 6.36170435/579.2327271 secs/batch = 0.5519s, grad.norm=0.32726750
  1120: 0 [ 1120/ 1499], train_loss/perplexity = 6.60577869/739.3553467 secs/batch = 0.5395s, grad.norm=0.40353474
  1125: 0 [ 1125/ 1499], train_loss/perplexity = 6.54308033/694.4223633 secs/bat

  1445: 0 [ 1445/ 1499], train_loss/perplexity = 6.31693411/553.8722534 secs/batch = 0.5748s, grad.norm=0.35198790
  1450: 0 [ 1450/ 1499], train_loss/perplexity = 6.31810999/554.5239258 secs/batch = 0.5382s, grad.norm=0.29216608
  1455: 0 [ 1455/ 1499], train_loss/perplexity = 6.54954863/698.9286499 secs/batch = 0.5433s, grad.norm=0.37018383
  1460: 0 [ 1460/ 1499], train_loss/perplexity = 6.58060026/720.9719849 secs/batch = 0.5425s, grad.norm=0.38409078
  1465: 0 [ 1465/ 1499], train_loss/perplexity = 6.81263685/909.2652588 secs/batch = 0.5421s, grad.norm=0.37625739
  1470: 0 [ 1470/ 1499], train_loss/perplexity = 6.41194677/609.0782471 secs/batch = 0.5395s, grad.norm=0.46633723
  1475: 0 [ 1475/ 1499], train_loss/perplexity = 6.37993383/589.8886719 secs/batch = 0.5380s, grad.norm=0.33547100
  1480: 0 [ 1480/ 1499], train_loss/perplexity = 6.52528381/682.1733398 secs/batch = 0.5355s, grad.norm=0.33582419
  1485: 0 [ 1485/ 1499], train_loss/perplexity = 6.23986483/512.7891846 secs/bat

...

  4682: 3 [  185/ 1499], train_loss/perplexity = 5.57537842/263.8493958 secs/batch = 0.5401s, grad.norm=0.46138558
  4687: 3 [  190/ 1499], train_loss/perplexity = 5.85907793/350.4009094 secs/batch = 0.5362s, grad.norm=0.49702737
  4692: 3 [  195/ 1499], train_loss/perplexity = 5.87798166/357.0877991 secs/batch = 0.5459s, grad.norm=0.48084006
  4697: 3 [  200/ 1499], train_loss/perplexity = 5.83228970/341.1388855 secs/batch = 0.5455s, grad.norm=0.58503973
  4702: 3 [  205/ 1499], train_loss/perplexity = 5.72810173/307.3852234 secs/batch = 0.5445s, grad.norm=0.44629699
  4707: 3 [  210/ 1499], train_loss/perplexity = 5.52109480/249.9084930 secs/batch = 0.5395s, grad.norm=0.50454909
  4712: 3 [  215/ 1499], train_loss/perplexity = 5.69145441/296.3242798 secs/batch = 0.5895s, grad.norm=0.55369788
  4717: 3 [  220/ 1499], train_loss/perplexity = 5.51014423/247.1867828 secs/batch = 0.5530s, grad.norm=0.49501511
  4722: 3 [  225/ 1499], train_loss/perplexity = 5.69811869/298.3056641 secs/bat

  5042: 3 [  545/ 1499], train_loss/perplexity = 5.30772686/201.8907776 secs/batch = 0.5370s, grad.norm=0.55924791
  5047: 3 [  550/ 1499], train_loss/perplexity = 5.75231600/314.9191589 secs/batch = 0.5431s, grad.norm=0.56208599
  5052: 3 [  555/ 1499], train_loss/perplexity = 5.54132462/255.0155792 secs/batch = 0.5413s, grad.norm=0.51101333
  5057: 3 [  560/ 1499], train_loss/perplexity = 5.93056488/376.3670654 secs/batch = 0.5470s, grad.norm=0.52870917
  5062: 3 [  565/ 1499], train_loss/perplexity = 5.78970432/326.9163513 secs/batch = 0.5393s, grad.norm=0.50941259
  5067: 3 [  570/ 1499], train_loss/perplexity = 5.75434971/315.5602722 secs/batch = 0.5432s, grad.norm=0.48101228
  5072: 3 [  575/ 1499], train_loss/perplexity = 5.77391195/321.7941284 secs/batch = 0.5389s, grad.norm=0.50528854
  5077: 3 [  580/ 1499], train_loss/perplexity = 5.55038595/257.3368530 secs/batch = 0.5426s, grad.norm=0.66021574
  5082: 3 [  585/ 1499], train_loss/perplexity = 5.44460630/231.5061188 secs/bat

  5402: 3 [  905/ 1499], train_loss/perplexity = 5.56944132/262.2875366 secs/batch = 0.5392s, grad.norm=0.58234221
  5407: 3 [  910/ 1499], train_loss/perplexity = 5.72072554/305.1262207 secs/batch = 0.5473s, grad.norm=0.57953745
  5412: 3 [  915/ 1499], train_loss/perplexity = 5.48584890/241.2536621 secs/batch = 0.5523s, grad.norm=0.59361750
  5417: 3 [  920/ 1499], train_loss/perplexity = 5.19138479/179.7172546 secs/batch = 0.5415s, grad.norm=0.46387133
  5422: 3 [  925/ 1499], train_loss/perplexity = 5.62708712/277.8515930 secs/batch = 0.5383s, grad.norm=0.65862906
  5427: 3 [  930/ 1499], train_loss/perplexity = 5.56869030/262.0906067 secs/batch = 0.5473s, grad.norm=0.57407534
  5432: 3 [  935/ 1499], train_loss/perplexity = 5.66202450/287.7305603 secs/batch = 0.5414s, grad.norm=0.52726960
  5437: 3 [  940/ 1499], train_loss/perplexity = 5.28040934/196.4502716 secs/batch = 0.5382s, grad.norm=0.62236583
  5442: 3 [  945/ 1499], train_loss/perplexity = 5.65296650/285.1360779 secs/bat

  5762: 3 [ 1265/ 1499], train_loss/perplexity = 5.59605455/269.3615417 secs/batch = 0.5438s, grad.norm=0.48422015
  5767: 3 [ 1270/ 1499], train_loss/perplexity = 5.57837152/264.6402893 secs/batch = 0.5966s, grad.norm=0.51380485
  5772: 3 [ 1275/ 1499], train_loss/perplexity = 5.59074068/267.9339905 secs/batch = 0.5399s, grad.norm=0.58012235
  5777: 3 [ 1280/ 1499], train_loss/perplexity = 5.22716475/186.2639465 secs/batch = 0.5368s, grad.norm=0.58040404
  5782: 3 [ 1285/ 1499], train_loss/perplexity = 5.67457438/291.3642883 secs/batch = 0.5413s, grad.norm=0.60756665
  5787: 3 [ 1290/ 1499], train_loss/perplexity = 5.61714792/275.1036377 secs/batch = 0.5387s, grad.norm=0.58219230
  5792: 3 [ 1295/ 1499], train_loss/perplexity = 5.49642706/243.8192291 secs/batch = 0.5413s, grad.norm=0.51811647
  5797: 3 [ 1300/ 1499], train_loss/perplexity = 5.68850279/295.4509277 secs/batch = 0.5385s, grad.norm=0.49984932
  5802: 3 [ 1305/ 1499], train_loss/perplexity = 5.56954527/262.3147888 secs/bat

  6121: 4 [  125/ 1499], train_loss/perplexity = 5.50282669/245.3845825 secs/batch = 0.5560s, grad.norm=0.68629920
  6126: 4 [  130/ 1499], train_loss/perplexity = 5.42908764/227.9411926 secs/batch = 0.5634s, grad.norm=0.53032947
  6131: 4 [  135/ 1499], train_loss/perplexity = 5.43758011/229.8852081 secs/batch = 0.5509s, grad.norm=0.57254535
  6136: 4 [  140/ 1499], train_loss/perplexity = 5.44690466/232.0388184 secs/batch = 0.5502s, grad.norm=0.60254824
  6141: 4 [  145/ 1499], train_loss/perplexity = 5.18459749/178.5015869 secs/batch = 0.5537s, grad.norm=0.55803508
  6146: 4 [  150/ 1499], train_loss/perplexity = 5.41601610/224.9810333 secs/batch = 0.5496s, grad.norm=0.55128151
  6151: 4 [  155/ 1499], train_loss/perplexity = 5.54189444/255.1609344 secs/batch = 0.5480s, grad.norm=0.52777064
  6156: 4 [  160/ 1499], train_loss/perplexity = 5.82739353/339.4726868 secs/batch = 0.6064s, grad.norm=0.57841212
  6161: 4 [  165/ 1499], train_loss/perplexity = 5.26996422/194.4090118 secs/bat

In [5]:
import importlib

In [6]:
importlib.reload(model)
model.TrainSentiModel(sess, saver, senti_train_graph, sentiReader, FLAGS.batch_size, FLAGS.batch_size)

data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
sentX shape: Tensor("Placeholder_1:0", shape=(?, 31, 21), dtype=int32)
sent_y, shape: Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
data_X shape: (20, 31, 21)
data_Y shape: (20, 3)
s

NameError: name 'logger' is not defined

In [6]:
importlib.reload(model)
model.TrainRDMModel(sess, rdm_train_graph, 0.7, 100)

Unknown char: ’
Word: that’s
Unknown char: ’
Word: said..he’s
Unknown char: ’
Word: they’re
Unknown char: "
Word: a"cleric
Unknown char: è
Word: ministère
Unknown char: ’
Word: l’intérieur
Unknown char: é
Word: l’intérieur
Unknown char: ’
Word: today’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wouldn’t
Unknown char: "
Word: been"justifying
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknow

NameError: name 'logger' is not defined

In [9]:
importlib.reload(model)
model.TrainCMModel(sess, rdm_train_graph, cm_train_graph, 0.3, 100)

in RL the begining
Unknown char: ’
Word: that’s
Unknown char: ’
Word: said..he’s
Unknown char: ’
Word: they’re
Unknown char: "
Word: a"cleric
Unknown char: è
Word: ministère
Unknown char: ’
Word: l’intérieur
Unknown char: é
Word: l’intérieur
Unknown char: ’
Word: today’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wouldn’t
Unknown char: "
Word: been"justifying
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: w░e░e░k
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: e░n░d
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Word: h░a░p░p░y
Unknown char: ░
Wor

Unknown char: ề
Word: iều
Unknown char: ò
Word: hòa
Unknown char: ầ
Word: trần
Unknown char: =
Word: massmedia=junkfood
Unknown char: é
Word: gérard
Unknown char: 🐸
Word: french🐸frog
Unknown char: é
Word: défendre
Unknown char: é
Word: démocratie
Unknown char: ä
Word: gefährlich
Unknown char: ’
Word: that’s
Unknown char: é
Word: célèbrités
Unknown char: è
Word: célèbrités
Unknown char: é
Word: célèbrités
Unknown char: è
Word: ramènent
Unknown char: ê
Word: même
Unknown char: "
Word: issues."i
Unknown char: ’
Word: today’s
Unknown char: ’
Word: today’s
Unknown char: "
Word: unemplymt."they"deserve
Unknown char: "
Word: unemplymt."they"deserve
Unknown char: ’
Word: it’s
Unknown char: ’
Word: it’s
Unknown char: "
Word: i"m
Unknown char: é
Word: protégés
Unknown char: é
Word: protégés
Unknown char: ’
Word: they’re
Unknown char: …
Word: display…as
Unknown char: …
Word: staff…publish
Unknown char: …
Word: week…1m
Unknown char: "
Word: important"...more
Unknown char: "
Word: the"religion
Unkn

Unknown char: é
Word: spéisiúl
Unknown char: ú
Word: spéisiúl
Unknown char: ó
Word: fóill
Unknown char: í
Word: shuíomh
Unknown char: é
Word: insécurit
Unknown char: ’
Word: md’s
Unknown char: é
Word: astérix
Unknown char: é
Word: résistant
Unknown char: è
Word: très
Unknown char: á
Word: están
Unknown char: ó
Word: expresión
Unknown char: é
Word: también
Unknown char: í
Word: crítica
Unknown char: ’
Word: can’t
Unknown char: ’
Word: it’s
Unknown char: "
Word: article"how
Unknown char: =
Word: it=a
Unknown char: é
Word: l'amérique
Unknown char: "
Word: you're*...."as
Unknown char: "
Word: faith"when
Unknown char: ö
Word: angehörigen
Unknown char: ü
Word: mitgefühl
Unknown char: ü
Word: für
Unknown char: ’
Word: it’s
Unknown char: é
Word: aéreos
Unknown char: ’
Word: aren’t
Unknown char: ’
Word: that’s
Unknown char: ’
Word: gov’t
Unknown char: é
Word: norvégien
Unknown char: é
Word: dégouts
Unknown char: ’
Word: it’s
Unknown char: ’
Word: you’re
Unknown char: ’
Word: allan’s
Unknown cha

Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: wolinski’s
Unknown char: ’
Word: don’t
Unknown char: é
Word: démocratie
Unknown char: é
Word: décidément
Unknown char: é
Word: décidément
Unknown char: ―
Word: conviction―just
Unknown char: ―
Word: conviction―just
Unknown char: ’
Word: don’t
Unknown char: ë
Word: dammartin-en-goël
Unknown char: ’
Word: you’re
Unknown char: ’
Word: it’s
Unknown char: ’
Word: they’re
Unknown char: ’
Word: aren’t
Unknown char: ’
Word: it’s
Unknown char: ’
Word: it’s
Unknown char: í
Word: víctimas
Unknown char: 😷
Word: daily😷😷😷your
Unknown char: 😷
Word: daily😷😷😷your
Unknown char: 😷
Word: daily😷😷😷your
Unknown char: =
Word: violations=dismissal
Unknown char: ü
Word: türkiye’de
Unknown char: ’
Word: türkiye’de
Unknown char: ş
Word: işi̇d
Unknown ch

Unknown char: é
Word: priorités
Unknown char: é
Word: déplac
Unknown char: â
Word: mankind.âll
Unknown char: =
Word: sharialaw=hamas=isis
Unknown char: =
Word: sharialaw=hamas=isis
Unknown char: =
Word: haram=hezbollah
Unknown char: "
Word: bold"?...trying
Unknown char: =
Word: lies=lies=lies
Unknown char: =
Word: lies=lies=lies
Unknown char: ’
Word: won’t
Unknown char: ’
Word: i’m
Unknown char: ’
Word: isn’t
Unknown char: ’
Word: us’s
Unknown char: ’
Word: isn’t
Unknown char: ’
Word: us’s
Unknown char: =
Word: suspects=more
Unknown char: é
Word: téméraire
Unknown char: é
Word: téméraire
Unknown char: ’
Word: i’d
Unknown char: ’
Word: wasn’t
Unknown char: è
Word: frère
Unknown char: é
Word: montée
Unknown char: í
Word: policía
Unknown char: í
Word: polícia
Unknown char: á
Word: austrália
Unknown char: é
Word: mantém
Unknown char: á
Word: vários
Unknown char: é
Word: reféns
Unknown char: ’
Word: won’t
Unknown char: ’
Word: he’s
Unknown char: ’
Word: he’s
Unknown char: "
Word: w/"reporte

Unknown char: =
Word: americans=full
Unknown char: ä
Word: väl
Unknown char: ä
Word: självklarhet
Unknown char: ’
Word: you’re
Unknown char: ’
Word: l’histoire
Unknown char: ’
Word: envoy’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: there’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: doesn’t
Unknown char: ’
Word: i’ve
Unknown char: ’
Word: that’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: i’d
Unknown char: ’
Word: it’s
Unknown char: "
Word: indication"of
Unknown char: ’
Word: i’m
Unknown char: ’
Word: i’m
Unknown char: ’
Word: people’s
Unknown char: ’
Word: let’s
Unknown char: ’
Word: i’ve
Unknown char: ’
Word: bush’s
Unknown char: ’
Word: bush’s
Unknown char: ’
Word: there’s
Unknown char: ’
Word: someone’s
Unknown char: ’
Word: it’s
Unknown char: ’
Word: there’s
Unknown char: ’
Word: someone’s
Unknown char: ’
Word: gun.it’s
Unknown char: ü
Word: sürüyor
Unknown char: ü
Word: sürüyor
Unknown char: é
Word: dénonce
Unknown char: "
Word: much"im
Unknown char: ’
Word: y

Unknown char: =
Word: murder=intentional
Unknown char: ’
Word: canada’s
Unknown char: ä
Word: jämföra
Unknown char: ö
Word: jämföra
Unknown char: å
Word: någon
Unknown char: ö
Word: förstår
Unknown char: å
Word: förstår
Unknown char: å
Word: sånt
Unknown char: ä
Word: här
Unknown char: ö
Word: för
Unknown char: ö
Word: förlorare
Unknown char: ä
Word: väldigt
Unknown char: é
Word: l'amérique
Unknown char: è
Word: derrière
Unknown char: ’
Word: i’d
Unknown char: ’
Word: there’s
Unknown char: ’
Word: doesn’t
Unknown char: ’
Word: it’s
Unknown char: ’
Word: don’t
Unknown char: ’
Word: can’t
Unknown char: ’
Word: you’re
Unknown char: ’
Word: i’m
Unknown char: ö
Word: möglich
Unknown char: ¨
Word: it¨s
Unknown char: â
Word: islâmico
Unknown char: ã
Word: são
Unknown char: ã
Word: estão
Unknown char: ’
Word: i’d
Unknown char: "
Word: war."once
Unknown char: "
Word: you"re
Unknown char: "
Word: in"jean
Unknown char: á
Word: barbárie
Unknown char: á
Word: más
Unknown char: ñ
Word: viñetas
Unkno

Unknown char: =
Word: dreadheads=savages
Unknown char: ’
Word: don’t
Unknown char: ’
Word: that’s
Unknown char: ’
Word: it’s
Unknown char: ’
Word: wouldn’t
Unknown char: ’
Word: i’m
Unknown char: ’
Word: doesn’t
Unknown char: ’
Word: i’m
Unknown char: ’
Word: it’s
Unknown char: ’
Word: i’m
Unknown char: ’
Word: i’m
Unknown char: ’
Word: it’s
Unknown char: ’
Word: that’s
Unknown char: â
Word: châtiment
Unknown char: é
Word: c'était
Unknown char: é
Word: véritable
Unknown char: è
Word: succès
Unknown char: =
Word: hispanicsasians=democrat
Unknown char: =
Word: christians=republicans
Unknown char: =
Word: christians=dems
Unknown char: ü
Word: mitgefühl
Unknown char: ö
Word: gehört
Unknown char: ö
Word: angehörigen
Unknown char: ̇
Word: i̇srailli
Unknown char: ü
Word: tümgeneral
Unknown char: ç
Word: koçavi
Unknown char: ü
Word: türkiye’de
Unknown char: ’
Word: türkiye’de
Unknown char: 
make: gun
Unknown char: 
rewardsun
Unknown char: 
puttingead
Unknown char: ’
Word: i’m
Unknown char: ’
W

KeyboardInterrupt: 